<a href="https://colab.research.google.com/github/madnani96/K-Means-Clustering-on-Healthcare-data-set/blob/master/Assignment_3_KMeans(Gaurav_Madnani%2C_Dhruv_Phansalkar).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment 3 (Part2) K-Means

Gaurav Madnani GXM200009

Dhruv Phansalkar DXP200001

In [ ]:
pip install requests

In [ ]:

! git clone https://github.com/madnani96/K-Means-Clustering-on-Healthcare-data-set.git
import math
import random as r
import string
import requests
import re

fatal: destination path 'K-Means-Clustering-on-Healthcare-data-set' already exists and is not an empty directory.


Preprocessing

In [ ]:
#datalink='https://drive.google.com/drive/u/0/folders/18fpCI18ZnBYZeaWcevAg7b4oLnnuJbHJ'
datalink='/content/K-Means-Clustering-on-Healthcare-data-set/gdnhealthcare.txt'
#def Preprocessing(datalink):
file = open(datalink, "r", encoding="utf8")
data=list(file)
#print(data)
length=len(data)-1
print(len(data))
tweetlist=[]
i=0
while(length>i):

  data[i]=data[i][50:]
  data[i]=re.sub(r"http\S+", "",data[i])
  data[i]=re.sub(r"www\S+", "",data[i])        
  data[i]=data[i].strip('\n')
  data[i]=" ".join(filter(lambda y:y[0]!='@',data[i].split()))
  data[i]=data[i].translate({ord(':'): None})
  data[i]=data[i].translate({ord('#'): None})
  data[i]=data[i].lower()
  data[i] = data[i].translate(str.maketrans('', '', string.punctuation))
  data[i] = " ".join(data[i].split())
  data[i]=data[i].split(' ')
  tweetlist.append(data[i])
  i+=1
file.close()
print("List of Tweets after Preprocessing:")
print(tweetlist)




2997
List of Tweets after Preprocessing:
[['are', 'you', 'a', 'member', 'of', 'the', 'network', 'sign', 'up', 'here', 'for', 'free', 'nhs', 'healthcare'], ['what', 'is', 'palliative', 'care', 'like', 'in', 'india', 'one', 'gp', 'trainee', 'explains', 'all'], ['most', 'viewed', 'this', 'week', 'i', 'loved', 'being', 'a', 'midwife', 'but', 'bullying', 'stress', 'and', 'fear', 'made', 'me', 'resign', 'viewsfromthenhsfrontline'], ['how', 'can', 'technology', 'improve', 'mental', 'health', 'waiting', 'times', 'mentalhealth', 'nhs'], ['in', 'case', 'you', 'missed', 'it', 'why', 'the', 'nhs', 'shouldn’t', 'hog', 'the', 'limelight', 'this', 'generalelection'], ['india', 'is', 'a', 'stark', 'example', 'of', 'how', 'commodifying', 'healthcare', 'can', 'lead', 'to', 'corruption'], ['popular', 'on', 'the', 'network', 'now', 'ive', 'seen', 'firsthand', 'how', 'palliative', 'care', 'in', 'india', 'is', 'compromised', 'by', 'privatisation'], ['are', 'there', 'any', 'topics', 'you', 'think', 'we', 'sh

**Utility** **Functions**

In [ ]:
#Sum of Squared Errors
def getSSE(clusters):
  totalerror=0
  for x in range(0,len(clusters)):
          for t in range(0,len(clusters[x])):
              totalerror += (clusters[x][t][1] * clusters[x][t][1])
  return totalerror
  
#Jaccard Distance
def calculateJaccardDistance(t1,t2):
  #intersection = set(tweet1).intersection(tweet2)
  #union = set().union(tweet1, tweet2)
  return 1 - (len(set(t1).intersection(t2)) / len(set().union(t1, t2)))

#Check Convergence
def checkconverence(prev,new):
  if len(new)!=len(prev):
    return False
  for x in range(len(new)):
    if " ".join(new[x]) != " ".join(prev[x]):
      return False
  return True

def centroidupdate(c):
  centroidlist=[]
  cl=0
  length=len(c)
  while(cl<length):
    dpdismin=[]
    totalminimumdistance=math.inf
    ctindex=-1

    for x in range(len(c[cl])):
      dpdismin.append([])
      totaldistance=0
      for y in range(len(c[cl])):
        if x!=y:
          if y<x:
            distance=dpdismin[y][x]
          else:
            distance=calculateJaccardDistance(c[cl][x][0],c[cl][y][0])
          totaldistance+=distance
          dpdismin[x].append(distance)
        else:
          dpdismin[x].append(0)
      if totalminimumdistance>totaldistance:
        ctindex=x
        totalminimumdistance=totaldistance
    centroidlist.append(c[cl][ctindex][0])    

    cl+=1
  return centroidlist
def clusterassignment(tweetlist,centroid):
  c=dict()
  t=0
  
  while(t<len(tweetlist)):
    minimumdistance=math.inf
    cindex=-1
    d=0
    while(d<len(centroid)):
    
      distance=calculateJaccardDistance(centroid[d],tweetlist[t])
      if tweetlist[t]==centroid[d]:
        cindex=d
        minimumdistance=0
        break
      if minimumdistance>distance:
        minimumdistance=distance
        cindex=d
      d+=1
    if minimumdistance==1:
      cindex=r.randint(0,len(centroid)-1)

    c.setdefault(cindex, []).append([tweetlist[t]])
    c.setdefault(cindex, [])[len(c.setdefault(cindex, [])) - 1].append(minimumdistance)
    t+=1
  return c


K Means Algorithm

In [ ]:
kvalues=[4,8,12,16,20]
count=1

for k in kvalues:
    print("\n")
    print("Experiment Number:",count, ",k value=",k)
    centroids=[]
    ct=0
    dictionary=dict()

    while ct<k:
      rdtweetidx=r.randint(0, len(tweetlist) - 1)
      if rdtweetidx not in dictionary:
        ct+=1
        dictionary[rdtweetidx]=True
        centroids.append(tweetlist[rdtweetidx])
    iterations=0
    previouscentroids=[]
    while (iterations < 30)and checkconverence(previouscentroids, centroids) == False :
      
      clusters=clusterassignment(tweetlist,centroids)
      previouscentroids=centroids
      centroids=centroidupdate(clusters)
      
      iterations+=1
    print("Number of Iterations:",iterations)
    if(iterations==30):
      print("Maximum Iterations Reached")
    else:
      print("Convergence Achieved")
    sumsqerror=getSSE(clusters)
    
    c=0
    while(c<len(clusters)):
      print("Cluster",(c+1) , ": ", str(len(clusters[c])) + " tweets") 
      c+=1
    print("Sum of square Error=",sumsqerror)
    count+=1
    



Experiment Number: 1 ,k value= 4
Number of Iterations: 2
Convergence Achieved
Cluster 1 :  345 tweets
Cluster 2 :  1111 tweets
Cluster 3 :  948 tweets
Cluster 4 :  592 tweets
Sum of square Error= 2293.1760659733386


Experiment Number: 2 ,k value= 8
Number of Iterations: 3
Convergence Achieved
Cluster 1 :  436 tweets
Cluster 2 :  390 tweets
Cluster 3 :  448 tweets
Cluster 4 :  278 tweets
Cluster 5 :  254 tweets
Cluster 6 :  398 tweets
Cluster 7 :  392 tweets
Cluster 8 :  400 tweets
Sum of square Error= 2192.7965036096098


Experiment Number: 3 ,k value= 12
Number of Iterations: 3
Convergence Achieved
Cluster 1 :  239 tweets
Cluster 2 :  775 tweets
Cluster 3 :  304 tweets
Cluster 4 :  174 tweets
Cluster 5 :  38 tweets
Cluster 6 :  149 tweets
Cluster 7 :  203 tweets
Cluster 8 :  341 tweets
Cluster 9 :  298 tweets
Cluster 10 :  123 tweets
Cluster 11 :  163 tweets
Cluster 12 :  189 tweets
Sum of square Error= 2180.4989463344555


Experiment Number: 4 ,k value= 16
Number of Iterations: 3
